<a href="https://colab.research.google.com/github/amitthakur2013/Coursera_Capstone/blob/master/Four_Square.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
import requests
import pandas as pd
import numpy as np
import random

!pip install geopy
from geopy.geocoders import Nominatim

from IPython.display import Image
from IPython.core.display import HTML

#transforming json files into pandas data frame
from pandas.io.json import json_normalize


In [131]:
!pip install folium
import folium
print('Folium installed')
print('Libraries Imported!')

Folium installed
Libraries Imported!


**Define Foursquare Credentials and Version**

In [132]:
CLIENT_ID='AFOZEA5RMASS1VRLPCC0RXUM04BT2E4LHPFCXUJJOGLII44V'
CLIENT_SECRET='GUQ3NRZ5AD33VBLEP31WLPHBXMMQD2D2ESSTX5CU1B5XS4RK'
VERSION="20191016"
LIMIT=30
print('Your Credentials:-')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET '+CLIENT_SECRET)

Your Credentials:-
CLIENT_ID: AFOZEA5RMASS1VRLPCC0RXUM04BT2E4LHPFCXUJJOGLII44V
CLIENT_SECRET GUQ3NRZ5AD33VBLEP31WLPHBXMMQD2D2ESSTX5CU1B5XS4RK


**Convert the addresss to its Latitue and Longitude values**

In [133]:
address='102 North End Ave, New York, NY'
#address='saltlake, kolkata, west bengal, IN'
geolocator=Nominatim(user_agent='foursquare_agent')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


**To Search for a specific venues category**

***Let's define a query to search for italian food***

In [134]:
search_query='Italian'
radius=500
print(search_query+'........OK!')

Italian........OK!


**Now let us define the url**

In [135]:
url='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,latitude,longitude,VERSION,search_query,radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=AFOZEA5RMASS1VRLPCC0RXUM04BT2E4LHPFCXUJJOGLII44V&client_secret=GUQ3NRZ5AD33VBLEP31WLPHBXMMQD2D2ESSTX5CU1B5XS4RK&ll=40.7149555,-74.0153365&v=20191016&query=Italian&radius=500&limit=30'

**Send the get reques and analyze the results**

In [136]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5da88442396de0002caf6a0f'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'primary': True,
      'shortName': 'Pizza'}],
    'delivery': {'id': '294544',
     'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
       'prefix': 'https://fastly.4sqi.net/img/general/cap/',
       'sizes': [40, 50]},
      'name': 'seamless'},
     'url': 'https://www.seamless.com/menu/harrys-italian-pizza-bar-225-murray-st-new-york/294544?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=294544'},
    'hasPerk': False,
    'id': '4fa862b3e4b0ebff2f749f06',
    'location': {'address': '225 Murray St',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',
     'distance'

In [137]:
import datetime
datetime.datetime.now()

datetime.datetime(2019, 10, 17, 15, 9, 55, 859630)

In [140]:
venues= results['response']['venues']
dataframe=json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",294544,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/harrys-italian-p...,False,4fa862b3e4b0ebff2f749f06,225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,Harry's Italian Pizza Bar,v-1571324994
1,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3232e219836c91c7bfde94,63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,Conca Cucina Italian Restaurant,v-1571324994
2,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,3fd66200f964a520f4e41ee3,124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,Ecco,v-1571324994


**Define informations of interest and filter dataframe**

In [142]:
#keep only columns containing venue name and containing information related to locations
filtered_columns=['name','categories']+[col for col in dataframe.columns if col.startswith('location.')] + ['id']
filtered_columns
dataframe_filtered=dataframe.loc[:,filtered_columns]
dataframe_filtered

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,id
0,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


In [143]:
dataframe_filtered.columns=[column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


In [148]:
# function that extracts the category of the venue
def get_category_type(row):
  try:
    categories_list=row['categories']
  except:
    categories_list=row['venue.categories']
  if len(categories_list)== 0:
    return None
  else:
    return categories_list[0]['name']
  
#filter the categoryfor each row

dataframe_filtered['categories']=dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


In [150]:
dataframe_filtered['name']

0          Harry's Italian Pizza Bar
1    Conca Cucina Italian Restaurant
2                               Ecco
Name: name, dtype: object

In [155]:
#generate map centered around the conrad hotel
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

/bin/bash: conda: command not found


In [0]:
venues_map

#Now add the italian restaurants as blue marks
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng,dataframe_filtered.categories):
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      color='blue',
      fill=True,
      fill_color='blue',
      fill_opacity=0.6
  ).add_to(venues_map)
  

In [159]:
#Display Map
venues_map

## **2. Explore a given Venue**

In [163]:
#Lets explore the closest italian restaurant
venue_id=dataframe_filtered.id[0] #ID of Harry's Italian Pizza bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=AFOZEA5RMASS1VRLPCC0RXUM04BT2E4LHPFCXUJJOGLII44V&client_secret=GUQ3NRZ5AD33VBLEP31WLPHBXMMQD2D2ESSTX5CU1B5XS4RK&v=20191016'

**Send Get Requests for Result**

In [164]:
result=requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '$$', 'priceTier': 2}],
    'name': 'Price',
    'summary': '$$',
    'type': 'price'},
   {'count': 7,
    'items': [{'displayName': 'Credit Cards',
      'displayValue': 'Yes (incl. American Express)'}],
    'name': 'Credit Cards',
    'summary': 'Credit Cards',
    'type': 'payments'},
   {'count': 1,
    'items': [{'displayName': 'Outdoor Seating', 'displayValue': 'Yes'}],
    'name': 'Outdoor Seating',
    'summary': 'Outdoor Seating',
    'type': 'outdoorSeating'},
   {'count': 8,
    'items': [{'displayName': 'Brunch', 'displayValue': 'Brunch'},
     {'displayName': 'Lunch', 'displayValue': 'Lunch'},
     {'displayName': 'Dinner', 'displayValue': 'Dinner'},
     {'displayName': 'Happy Hour', 'displayValue': 'Happy Hour'}],
    'name': 'Menus',
    'summary': 'Happy Hour, Brunch & more',
    'type': 'serves'},
   {'count': 5,
    'items': [{'displayName': 'Bee

In [165]:
#Get the venues of overall rating

try:
  print(result['response']['venue']['rating'])
except:
  print("This venue has not been rated yet!")

7.0


Lets check other retaurant rating

In [169]:
venue_id=dataframe_filtered.id[1]
url='https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4f3232e219836c91c7bfde94?client_id=AFOZEA5RMASS1VRLPCC0RXUM04BT2E4LHPFCXUJJOGLII44V&client_secret=GUQ3NRZ5AD33VBLEP31WLPHBXMMQD2D2ESSTX5CU1B5XS4RK&v=20191016'

In [172]:
result=requests.get(url).json()
try:
  print('Rating ',result['response']['venue']['rating'])
except:
  print('This venue has not been rated yet')

This venue has not been rated yet


**Lets check the 3rd Restaurant**

In [174]:
venue_id=dataframe_filtered.id[2]
url='https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id,CLIENT_ID,CLIENT_SECRET,VERSION)
url

'https://api.foursquare.com/v2/venues/3fd66200f964a520f4e41ee3?client_id=AFOZEA5RMASS1VRLPCC0RXUM04BT2E4LHPFCXUJJOGLII44V&client_secret=GUQ3NRZ5AD33VBLEP31WLPHBXMMQD2D2ESSTX5CU1B5XS4RK&v=20191016'

In [175]:
result=requests.get(url).json()
try:
  print('Rating :',result['response']['venue']['rating'])
except:
  print('This restaurant has not been rated yet!')

Rating : 7.9


In [178]:
print("Total number of tips =",result['response']['venue']['tips']['count'])

Total number of tips = 17


Since this restaurant has a slightly better rating, let us explore this more..

In [179]:
#Get the venue's Tips
LIMIT=15
url='https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id,CLIENT_ID,CLIENT_SECRET,VERSION,LIMIT)
url

'https://api.foursquare.com/v2/venues/3fd66200f964a520f4e41ee3/tips?client_id=AFOZEA5RMASS1VRLPCC0RXUM04BT2E4LHPFCXUJJOGLII44V&client_secret=GUQ3NRZ5AD33VBLEP31WLPHBXMMQD2D2ESSTX5CU1B5XS4RK&v=20191016&limit=15'

In [186]:
result=requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5da89e15cc7d41002b91d6ad'},
 'response': {'tips': {'count': 17,
   'items': [{'agreeCount': 3,
     'authorInteractionType': 'liked',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'disagreeCount': 0,
     'id': '5ab1cb46c9a517174651d3fe',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'todo': {'count': 0},
     'type': 'user',
     'user': {'firstName': 'Nick',
      'gender': 'male',
      'id': '484542633',
      'lastName': 'El-Tawil',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg'}}}]}}}

In [187]:
#Get tips and list of Associated Features

tips = result['response']['tips']['items']

tip = result['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

**Format Column and display all tips**

In [190]:
#pd.set_option('display.max_colwidth', -1)

tips_df =json_normalize(tips)

#columns to keep

filtered_columns=['text','agreeCount','disagreeCount','id','user.firstName','user.lastName','user.gender','user.id']
tips_filtered=tips_df.loc[:, filtered_columns]

#display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",3,0,5ab1cb46c9a517174651d3fe,Nick,El-Tawil,male,484542633


**Search a foursquare User**

In [191]:
user_id='484542633'

url='https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id,CLIENT_ID,CLIENT_SECRET,VERSION)
url

'https://api.foursquare.com/v2/users/484542633?client_id=AFOZEA5RMASS1VRLPCC0RXUM04BT2E4LHPFCXUJJOGLII44V&client_secret=GUQ3NRZ5AD33VBLEP31WLPHBXMMQD2D2ESSTX5CU1B5XS4RK&v=20191016'

In [192]:
results=requests.get(url).json()
user_data=results['response']['user']

user_data.keys()

dict_keys(['id', 'firstName', 'lastName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'type', 'mayorships', 'checkins', 'lists', 'lenses'])

In [193]:
print('First Name: '+user_data['firstName'])
print('Last Name: '+user_data['lastName'])
print('Home City: '+user_data['homeCity'])

First Name: Nick
Last Name: El-Tawil
Home City: New York, NY


In [194]:
user_data['tips']

{'count': 238}

**Get User's Tips**

In [205]:
LIMIT=15
url='https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id,CLIENT_ID,CLIENT_SECRET,VERSION,LIMIT)

#send Get request and get user's request
results=requests.get(url).json()
tips=results['response']['tips']['items']

tips_df=json_normalize(tips)

#filter columns

filtered_columns=['text','agreeCount','disagreeCount','id']
tips_filtered=tips_df.loc[:, filtered_columns]

#display user's tips
tips_filtered


,text,agreeCount,disagreeCount,id
0,"The best! I’m especially fond of the salmon burger, but I’ve had half of the menu and never been disappointed. There’s a reason this place is well known even outside of the Village!",1,0,5aec594b1f7440002c138612
1,I used to down a pint of chocolate like it was nothing back when I was bulking. Highly recommended!,1,0,5accc9f66fa81f196724807b
2,They serve coffee!!!!!!,1,0,5accc98c0313204c9d7ec157
3,"I’m a fan. In fact, I’m such a big fan, I want Taim to hire me to be their spokesman. Kind of like the Arabic Jared Fogle - but without the kid stuff.",1,0,5accbf033abcaf09a24612a0
4,The linguine with clams is on point 👌,1,0,5accbe3a911fc423730f3ed3
5,"Great for a quick, cheap lunch! Shorter lines than Chipotle too👌",1,0,5acbecb86fa81f1967e019b0
6,"Quick, cheap lunch that tastes good! Way shorter line than Chipotle, too.",1,0,5acbec70a0215b732e264fe8
7,You’re not a real New Yorker until you’ve shame-ordered Insomnia Cookies for delivery at 3am,1,0,5acbbd4eb1538e45373b07f5
8,Good for you yet still tasty! Clean green protein is my go-to after I hit the gym 💪,2,0,5acbbcda01235808d5d6dc75
9,Coffee game on point,1,0,5acbbb1501235808d5d6525e


**Let's get the venue for the tip with the greatest number of agree counts**

In [206]:
tip_id = '5ab5575d73fe2516ad8f363b'

url="https://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}".format(tip_id,CLIENT_ID,CLIENT_SECRET,VERSION)
url

#send GET Request and examine results
result=requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

Cowgirl
{'address': '519 Hudson St', 'crossStreet': 'at W 10th St', 'lat': 40.73373338282062, 'lng': -74.0062998849649, 'labeledLatLngs': [{'label': 'display', 'lat': 40.73373338282062, 'lng': -74.0062998849649}], 'postalCode': '10014', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['519 Hudson St (at W 10th St)', 'New York, NY 10014', 'United States']}


In [207]:
#Get User's friends
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

""


In [208]:
user_data

{'bio': 'https://www.tawil.team/nick-el-tawil/',
 'canonicalUrl': 'https://foursquare.com/nickeltawil',
 'checkins': {'count': 1, 'items': []},
 'contact': {},
 'firstName': 'Nick',
 'friends': {'count': 0,
  'groups': [{'count': 0,
    'items': [],
    'name': 'Other friends',
    'type': 'others'}]},
 'gender': 'male',
 'homeCity': 'New York, NY',
 'id': '484542633',
 'lastName': 'El-Tawil',
 'lenses': [],
 'lists': {'count': 2,
  'groups': [{'count': 0, 'items': [], 'type': 'created'},
   {'count': 0, 'items': [], 'type': 'followed'},
   {'count': 2,
    'items': [{'canonicalUrl': 'https://foursquare.com/nickeltawil/list/todos',
      'collaborative': False,
      'description': '',
      'editable': False,
      'id': '484542633/todos',
      'listItems': {'count': 0},
      'name': "Nick's Saved Places",
      'public': True,
      'type': 'todos',
      'url': '/nickeltawil/list/todos'},
     {'canonicalUrl': 'https://foursquare.com/nickeltawil/list/venuelikes',
      'collaborat

In [209]:
Image(url='https://igx.4sqi.net/img/user/300x300/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg')

**Explore the location**

In [0]:
latitude = 40.715337
longitude = -74.008848

In [211]:
url="https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,latitude,longitude,VERSION,radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=AFOZEA5RMASS1VRLPCC0RXUM04BT2E4LHPFCXUJJOGLII44V&client_secret=GUQ3NRZ5AD33VBLEP31WLPHBXMMQD2D2ESSTX5CU1B5XS4RK&ll=40.715337,-74.008848&v=20191016&radius=500&limit=15'

In [0]:
import requests

In [214]:
results=requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))


'There are 15 around Ecco restaurant.'

In [215]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4af5d65ff964a52091fd21e3-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/furniture_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1f8941735',
    'name': 'Furniture / Home Store',
    'pluralName': 'Furniture / Home Stores',
    'primary': True,
    'shortName': 'Furniture / Home'}],
  'id': '4af5d65ff964a52091fd21e3',
  'location': {'address': '57 Warren St',
   'cc': 'US',
   'city': 'New York',
   'country': 'United States',
   'crossStreet': 'Church St',
   'distance': 73,
   'formattedAddress': ['57 Warren St (Church St)',
    'New York, NY 10007',
    'United States'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.71482437714839,
     'lng': -74.00940425461492}],
   'lat': 40.71482437714839,
   'lng': -74.00940425461492,
   'neighborhood': 'Tribeca',
   

In [216]:
dataframe=json_normalize(items)

#filter columns
filtered_columns=['venue.name','venue.categories']+[col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered=dataframe.loc[:, filtered_columns]
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Korin,Furniture / Home Store,57 Warren St,US,New York,United States,Church St,73,"[57 Warren St (Church St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}]",40.714824,-74.009404,Tribeca,10007,NY,4af5d65ff964a52091fd21e3
1,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,US,New York,United States,btwn Greenwich St & W Broadway,202,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",40.714788,-74.011132,NaN,10007,NY,54148bc6498ea7bb8c05b70a
2,Takahachi Bakery,Bakery,25 Murray St,US,New York,United States,at Church St,187,"[25 Murray St (at Church St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.713652845301894, 'lng': -74.0088038953017}]",40.713653,-74.008804,NaN,10007,NY,4c154c9a77cea593c401d260
3,Chambers Street Wines,Wine Shop,148 Chambers St,US,New York,United States,btwn West Broadway & Hudson St,88,"[148 Chambers St (btwn West Broadway & Hudson St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.715773063928374, 'lng': -74.00971823312332}]",40.715773,-74.009718,NaN,10007,NY,4adcf23cf964a520cc6221e3
4,Takahachi,Sushi Restaurant,145 Duane St,US,New York,United States,btwn W Broadway & Church St,146,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.71652647412374, 'lng': -74.00810108466207}]",40.716526,-74.008101,NaN,10013,NY,4a8f2f39f964a520471420e3
5,Philip Williams Posters,Antique Shop,122 Chambers St,US,New York,United States,NaN,8,"[122 Chambers St, New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71528423132827, 'lng': -74.00878093952018}]",40.715284,-74.008781,NaN,10007,NY,4b747291f964a52042dd2de3
6,Heyday,Spa,92 Reade St,US,New York,United States,NaN,86,"[92 Reade St, New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.715598486687675, 'lng': -74.00788227511288}]",40.715598,-74.007882,NaN,10013,NY,57ad129c498e05b086594d72
7,Equinox Tribeca,Gym,54 Murray Street,US,New York,United States,at W Broadway,154,"[54 Murray Street (at W Broadway), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71409860726041, 'lng': -74.0096857179283}]",40.714099,-74.009686,NaN,10007,NY,4a6e331af964a52031d41fe3
8,Mulberry & Vine,Café,73 Warren St,US,New York,United States,btwn W Broadway & Greenwich St,117,"[73 Warren St (btwn W Broadway & Greenwich St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71517693966315, 'lng': -74.01022747778285}]",40.715177,-74.010227,NaN,10007,NY,5171b5cc011cef9833bbb787
9,Nish Nūsh,Falafel Restaurant,88 Reade St,US,New York,United States,at Church St,97,"[88 Reade St (at Church St), New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.71553710116416, 'lng': -74.00772452925565}]",40.715537,-74.007725,NaN,10013,NY,50ba9119e4b071a4bae6dc10


In [218]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [219]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5da8b551cad1b6002c425289'},
 'response': {'venues': []}}

In [0]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [221]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

In [0]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [223]:
venues_map

'Cannot generate visual as no trending venues are available at the moment!'